In [1]:
from __future__ import division
from __future__ import print_function
import os
import sys
import pints
import pints.io
import numpy as np
import myokit
import matplotlib.pyplot as plt
#%matplotlib inline
# Load beattie model and prior
sys.path.append(os.path.abspath('models_forward'))
import beattie


In [2]:
cell = 5
root = os.path.abspath('sine-wave-data')#os.path.realpath(os.path.join('..'))
print(root)
data_file = os.path.join(root, 'cell-' + str(cell) + '.csv')
protocol_file = os.path.join(root,'steps.mmt')

/home/sanosh/pints-ikr/sine-wave-data


In [3]:
temperature = beattie.temperature(cell)
lower_conductance = beattie.conductance_limit(cell)


In [4]:
root = os.path.abspath('cmaes_results')
parameter_filename = os.path.join(root, 'cell-' + str(cell) + '-solution.txt')
print('Loading parameters from ' + parameter_filename)
with open(parameter_filename, 'r') as f:
    initial_parameters = [float(x) for x in f.readlines()]
initial_parameters = np.array(initial_parameters)

Loading parameters from /home/sanosh/pints-ikr/cmaes_results/cell-5-solution.txt


In [5]:
protocol = myokit.load_protocol(protocol_file)
log = myokit.DataLog.load_csv(data_file).npview()
time = log.time()
current = log['current']
voltage = log['voltage']
del(log)
sigma_noise = np.std(current[:2000], ddof=1)
print('Applying capacitance filtering')
time, voltage, current = beattie.capacitance(
    protocol, 0.1, time, voltage, current)

Applying capacitance filtering


In [6]:
model = beattie.BeattieModel(protocol, temperature, sine_wave=True)

In [7]:
problem = pints.SingleOutputProblem(model, time, current)

In [8]:
log_likelihood = pints.KnownNoiseLogLikelihood(problem, sigma_noise)
log_prior = beattie.BeattieLogPrior(lower_conductance)
#log_posterior = pints.LogPosterior(log_likelihood, log_prior)

In [9]:
a=log_prior.sample()
a=np.vstack((a,log_prior.sample()))
a.shape
current.shape
log_prior(log_prior.sample())

0

In [10]:
import theano.tensor as tt
from theano.compile.ops import as_op
import pymc3 as pm
from pymc3.theanof import floatX


@as_op(itypes=[tt.dscalar], otypes=[tt.dmatrix])
def randsample(samples):
    samples=int(samples)
    sam = log_prior.sample()
    for i in range(samples-1):
        sam = np.vstack((sam,log_prior.sample()))
    
    return sam


@as_op(itypes=[tt.dvector], otypes=[tt.dvector])
def log_p(value):
    lp = float(log_prior(value))
    return np.array([lp])

class BeattiePMC(pm.Continuous):
    def __init__(self, lower, upper, *args, **kwargs):
        super(BeattiePMC, self).__init__(*args, **kwargs)
        self.lower = lower = tt.as_tensor_variable(floatX(lower))
        self.upper = upper = tt.as_tensor_variable(floatX(upper))
        self.mean = (upper + lower) / 2.
        self.median = self.mean

    def random(self, point=None, size=None, repeat=None):

        
        samples=tt.as_tensor_variable(floatX(size))
        r=randsample(samples)
        
        return r
    
    def logp(self, value):

        return log_p(value)
    

Make sure that Python modules _winreg, win32api or win32con are installed.


In [11]:
startpoint=[initial_parameters for _ in range(100)]
startpoint=np.stack(startpoint,axis=0)
kylie_cmaes=np.array( [2.26137846758451367e-04,
 6.99130764903859586e-02,
 3.44958226788683007e-05,
 5.46121826895913237e-02,
 8.73242841545401188e-02,
 8.93017989292029316e-03,
 5.14883265963462632e-03,
 3.15620982414665449e-02,
 1.52436968881218465e-01])


In [12]:
@as_op(itypes=[tt.dvector], otypes=[tt.dvector])
def Beattie_ode_sol(param):
    
    
    sol=model.simulate(param,time)
    
    return sol


In [13]:
from pymc3.step_methods import smc
from tempfile import mkdtemp
import theano

theano.config.floatX = 'float64'
basic_model = pm.Model()
n_steps = 100 # Number of MCMC steps for each particle
n_chains = 500 # Number of particles (chains)

with basic_model:

    # Priors for unknown model parameters
    
    

    p = BeattiePMC('p', lower=0, upper=10, transform=None, shape=9)
    
    #p = pm.Gamma('p', alpha=2,beta=1, transform=None, shape=9)


    # Forward model
    
    mu = Beattie_ode_sol(p)
    
    # Likelihood (sampling distribution) of observations
    
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=sigma_noise, observed=current)
    #sd_print = tt.printing.Print('p')(p)
    
    test_folder = mkdtemp(prefix='SMC_TEST')
    
     # Initial points for each of the chains
    #startsmc=[{'a':np.random.uniform(0,15,1),'b':np.random.uniform(0,15,1),
    #        'c':np.random.uniform(2.5,3.5,1),'sigma':np.random.uniform(0,2)} for _ in range(n_chains)]
    startsmc=[{'p':kylie_cmaes+np.random.uniform(0,1e-5,9)} for _ in range(n_chains)]
    
    # Call SMC sampler
    #step=pm.Metropolis()
    #trace=pm.sample(50000,step=step,tune=True,chains=1,tune_interval=1000,cores=10,start=startsmc,progressbar=True)
    trace = smc.sample_smc(samples=5000,n_chains=n_chains,start=startsmc,progressbar=True,
                        homepath=test_folder,stage=0,cores=10,threshold=1,random_seed=20)
    

/home/sanosh/pymc3/pymc3/step_methods/smc.py:491: UserWarning: Warning: SMC is an experimental step method, and not yet recommended for use in PyMC3!
  warnings.warn(EXPERIMENTAL_WARNING)
/home/sanosh/pymc3/pymc3/step_methods/smc.py:118: UserWarning: Warning: SMC is an experimental step method, and not yet recommended for use in PyMC3!
  warnings.warn(EXPERIMENTAL_WARNING)


In [26]:
pm.plot_posterior(trace,color='LightSeaGreen')
plt.show()

In [ ]:
pm.summary(trace, ['p'])

In [ ]:
kylie_cmaes+np.random.uniform(0,1e-9,9)

In [27]:
np.log(basic_model.marginal_likelihood)

-37.30790445176793

In [ ]:
params=np.array([trace.get_values('p')])
params.shape

In [25]:
pplot.pairwise(params[0],opacity=1)
plt.show()

ValueError: `samples` must be of shape (n_sample, n_param)

In [71]:
basic_model

NameError: name 'basic_model' is not defined

In [12]:
df=pm.trace_to_dataframe(trace)
df.to_csv('sine-samples500.csv')

NameError: name 'trace' is not defined

In [21]:
params=np.array([trace.get_values('p')])[0]
new_values = []
for ind in range(len(params)):
    ppc_sol=model.simulate(params[ind], time)
    new_values.append(ppc_sol)
new_values = np.array(new_values)
mean_values = np.mean(new_values, axis=0)
new_values.shape
plt.figure()
plt.subplot(2,1,1)
plt.plot(time, voltage, color='orange', label='measured voltage')
plt.xlim(0,8000)
plt.subplot(2,1,2)
plt.plot(time, current,'--', color='blue',lw=1.5, label='measured current')
#plt.plot(time, new_values[0], color='#1f77b4', alpha=0.05, label='inferred currents')
#for v in new_values[1:]:
#    plt.plot(time, v, color='#1f77b4', alpha=0.05)
plt.plot(time, mean_values, color='seagreen', lw=1, label='mean of inferred current')
plt.xlim(0,8000)
plt.legend()
plt.show()
print('Done')

Done


In [14]:
import pandas as pd
df=pd.read_csv('sine-samples500.csv')
param=df.values

IOError: File sine-samples500.csv does not exist

In [65]:
param.shape[1]

10

In [69]:
import seaborn as sns; sns.set(color_codes=True)
sns.kdeplot(param[:,1])
plt.show()

In [42]:
plt.title('SMC posterior sine wave')
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.hist(param[:,i+1],50)
    plt.xlabel(('P'+str(i+1)))
    plt.ylabel('frequency')

plt.show()

In [39]:
import pints.plot as pplot
params=[param[:,1:10]]
pplot.trace(params)
plt.show()

In [40]:
pplot.pairwise(params[0],opacity=1)
plt.show()

In [43]:
params=param[:,1:10]

In [63]:
new_values = []
for ind in range(1000):
    ppc_sol=model.simulate(params[ind], time)
    new_values.append(ppc_sol)
new_values = np.array(new_values)
mean_values = np.mean(new_values, axis=0)
new_values.shape
plt.figure()
plt.subplot(3,1,1)
plt.plot(time, voltage, color='orange', label='measured voltage')
plt.xlim(0,8000)
plt.legend()
plt.subplot(3,1,2)
plt.plot(time, current,'--', color='blue',lw=1.5, label='measured current')
plt.plot(time, mean_values, color='SeaGreen', lw=1, label='mean of inferred current')
plt.xlim(0,8000)
plt.legend()
plt.subplot(3,1,3)
plt.plot(time[-40000:], current[-40000:],'--', color='blue',lw=1.5, label='measured current blow-up')
plt.plot(time[-40000:], mean_values[-40000:], color='SeaGreen', lw=1, label='mean of inferred current blow-up')
plt.xlim(4000,8000)
plt.legend()
plt.show()
print('Done')

Done


In [56]:
plt.plot(time[-40000:], current[-40000:],'--', color='blue',lw=1.5, label='measured current blow-up')

plt.show()